In [ ]:
#dataset =  https://www.kaggle.com/competitions/digit-recognizer

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

2024-08-22 18:49:05.285635: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 18:49:05.285764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 18:49:05.455590: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
x_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
#train.head()

Note the input is a Pandas DataFrame. We need to convert it to a numpy in order to tensorflow convert it to a tensor and properly handle the calculations in the network.

In [4]:
x_train = train.drop(['label'], axis=1)
y_train = train["label"]

x_train, x_test = np.array(x_train), np.array(x_test)
x_train.shape, x_test.shape

((42000, 784), (28000, 784))

In [5]:
x_train.shape, x_test.shape = (42000, 28,28), (28000, 28,28)
x_train[0].shape

(28, 28)

In [6]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
x_train.shape, x_test.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [7]:
x_train = (x_train - np.min(x_train)) / (np.max(x_train) - np.min(x_train))
x_test = (x_test - np.min(x_test)) / (np.max(x_test) - np.min(x_test))

#x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=1)

In [8]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=35,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.25,
    zoom_range=0.25,
    fill_mode='nearest'
)
datagen.fit(x_train)


augmented_data = datagen.flow(x_train, y_train, batch_size=len(x_train))
augmented_x_train, augmented_y_train = augmented_data[0]

combined_x_train = np.concatenate((x_train, augmented_x_train))
combined_y_train = np.concatenate((y_train, augmented_y_train))
print(combined_x_train.shape, combined_y_train.shape)

(84000, 28, 28, 1) (84000,)


In [9]:
#l2_reg_1 = tf.keras.regularizers.L2(l2=0.0095)
#l2_reg_2 = tf.keras.regularizers.L2(l2=0.012)
l2_reg_1 = tf.keras.regularizers.L2(l2=0.0)
l2_reg_2 = tf.keras.regularizers.L2(l2=0.0)

model = tf.keras.Sequential([
        #1st stacked conv layer
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    
        #1st pooling layer
    tf.keras.layers.AveragePooling2D(pool_size=4),
    
        #2nd stacked conv layer
    tf.keras.layers.Conv2D(filters=192, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2_reg_1),
    tf.keras.layers.Conv2D(filters=224, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=224, kernel_size=3, padding='same', activation='relu'),
    
        #2nd pooling layer
    tf.keras.layers.MaxPool2D(pool_size=3),
    
        #3rd stacked conv layer
    tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2_reg_2),
    tf.keras.layers.Conv2D(filters=272, kernel_size=3, padding='same', activation='relu'),
    
        #3rd pooling layer
    tf.keras.layers.MaxPool2D(pool_size=2),
    
        #fully-connect layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    
        #output layer
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
#model.summary()
model.fit(combined_x_train, combined_y_train, epochs=30, 
          #validation_data=(x_val, y_val), 
          batch_size=32)

Epoch 1/30
  14/2625 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - accuracy: 0.2002 - loss: 2.2069   

I0000 00:00:1724352712.488253     109 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2625/2625 ━━━━━━━━━━━━━━━━━━━━ 34s 8ms/step - accuracy: 0.8519 - loss: 0.4578
Epoch 2/30
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 33s 8ms/step - accuracy: 0.9708 - loss: 0.0984
Epoch 3/30
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9793 - loss: 0.0693
Epoch 4/30
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9846 - loss: 0.0539
Epoch 5/30
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9869 - loss: 0.0430
Epoch 6/30
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9897 - loss: 0.0369
Epoch 7/30
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9910 - loss: 0.0310
Epoch 8/30
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9923 - loss: 0.0247
Epoch 9/30
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9937 - loss: 0.0202
Epoch 10/30
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9954 - loss: 0.0159
Epoch 11/30
2625/2625 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9952 - loss: 0.0153
Epoch 12/30
2625/2625 ━━━━━━━━

In [10]:
predicted_labels = np.argmax(model.predict(x_test), axis=-1)

sub = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
sub['Label'] = predicted_labels
sub.to_csv('output.csv', index=False)

875/875 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [11]:
import pickle
Pkl_Filename = "model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)